# Appendix: Import needed modules 

In [1]:
import re, collections
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from spacy.en import English ##Note you'll need to install Spacy and download its dependencies
parser = English()
import string

In [2]:
# A custom stoplist
STOPLIST = set(stopwords.words('english') + ["n't", "'s", "'m", "ca"] + list(ENGLISH_STOP_WORDS))
# List of symbols we don't care about
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---", "...", "“", "”", "'ve"]

In [3]:
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(open('C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/bigtext.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [10]:
# A custom function to clean the text before sending it into the vectorizer
def cleanText(text):
    # get rid of newlines
    text = text.strip().replace("\n", " ").replace("\r", " ")
    
    # replace twitter @mentions
#     mentionFinder = re.compile(r"@[a-z0-9_]{1,15}", re.IGNORECASE)
#     text = mentionFinder.sub("@MENTION", text)
    text = re.sub('[^a-zA-Z0-9 ]','',text)
    # replace HTML symbols
    text = text.replace("&amp;", "and").replace("&gt;", ">").replace("&lt;", "<")
    
    # lowercase
    text = text.lower()
#     text = correct(text)
    return text

# A custom function to tokenize the text using spaCy
# and convert to lemmas
def tokenizeText(sample):

    # get the tokens using spaCy
    tokens = parser(str(TextBlob(sample).correct()))

    # lemmatize
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas

    # stoplist the tokens
    tokens = [tok for tok in tokens if tok not in STOPLIST]

    # stoplist symbols
    tokens = [tok for tok in tokens if tok not in SYMBOLS]

    # remove large strings of whitespace
    while "" in tokens:
        tokens.remove("")
    while " " in tokens:
        tokens.remove(" ")
    while "\n" in tokens:
        tokens.remove("\n")
    while "\n\n" in tokens:
        tokens.remove("\n\n")

    return tokens

In [11]:
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

import pandas as pd
df = pd.read_csv('C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/sentence_raw.csv',encoding = "ISO-8859-1")

# Download data and munge munge

In [12]:
df2 = df[['sentence','category']]

In [13]:
##Collapse some
df2[df2['category']=="wait"] = "service" 
df2[df2['category']=="value"] = "overall"

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\pandas\core\frame.py:2363: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.ix._setitem_with_indexer(indexer, value)
C:\Anaconda3\lib\site-packages\pandas\core\frame.py:2343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_array(key, value)
C:\Ana

In [14]:
df2.category.value_counts()/df2.shape[0]

food        0.414732
overall     0.263298
others      0.138862
service     0.123872
ambiance    0.059236
Name: category, dtype: float64

In [15]:
df2.sentence = df2.sentence.apply(cleanText)

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


# Feature Generation

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.decomposition import TruncatedSVD
import pickle

hashvectorizer = HashingVectorizer(ngram_range=(1,3),tokenizer=tokenizeText)
vectorizer = CountVectorizer(ngram_range=(1,3),min_df=3, max_features=5000,tokenizer=tokenizeText)
tfvectorizer = TfidfVectorizer(ngram_range=(1,3),min_df = 3,max_features=5000,tokenizer=tokenizeText)

In [18]:
##Hashing Vectorizer (slow)
hashfeature = hashvectorizer.fit_transform(df2.sentence)
hashfeature = pickle.dumps(hashfeature)
featuredf_hash = pd.DataFrame(hashfeature.A)
# df3_hash = pd.concat((df2,featuredf_hash),axis=1)
# print(df3_hash.info())
# df3_hash.to_pickle("df3_hash.pkl")

In [19]:
## Gets the count of each word in each sentence (Count Vectorizer)
countfeature = vectorizer.fit_transform(df2.sentence)
# lsa_count = TruncatedSVD(n_components=50,n_iter=100).fit_transform(countfeature)
# lsa_count_df = pd.DataFrame(lsa_count)
featuredf_count = pd.DataFrame(countfeature.A, columns=vectorizer.get_feature_names())
df3_count = pd.concat((df2,featuredf_count),axis=1)
# df3_lsa_count = pd.concat((df2,lsa_count_df),axis=1)
print(df3_count.info())
# print(df3_lsa_count.info())
df3_count.to_pickle("df3_count.pkl")
# df3_lsa_count.to_pickle("df3_lsa_count.pkl")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24816 entries, 0 to 24815
Columns: 5002 entries, sentence to zucchini
dtypes: int64(5000), object(2)
memory usage: 947.0+ MB
None


In [20]:
##TFIDF Vectorizer (slow)
tffeature = tfvectorizer.fit_transform(df2.sentence)
featuredf_tf = pd.DataFrame(tffeature.A, columns=tfvectorizer.get_feature_names())
# lsa_tf = TruncatedSVD(n_components=50,n_iter=100).fit_transform(tffeature)
# lsa_tf_df = pd.DataFrame(lsa_tf)
df3_tf = pd.concat((df2,featuredf_tf),axis=1)
# df3_lsa_tf = pd.concat((df2,lsa_tf_df),axis=1)
print(df3_tf.info())
# print(df3_lsa_tf.info())
df3_tf.to_pickle("df3_tf.pkl")
# df3_lsa_tf.to_pickle("df3_lsa_tf.pkl")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24816 entries, 0 to 24815
Columns: 5002 entries, sentence to zucchini
dtypes: float64(5000), object(2)
memory usage: 947.0+ MB
None


In [21]:
##Pickle vocab
count_dict = vectorizer.get_feature_names()
tf_dict = tfvectorizer.get_feature_names()
count_dict = pickle.dumps(count_dict)
tf_dict = pickle.dumps(tf_dict)


# Building the model and in-training-set accuracy

In [2]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split 
df3_count = pd.read_pickle("df3_count.pkl")
df3_tf = pd.read_pickle("df3_tf.pkl")

In [8]:
extrasample_df = df3_count[df3_count.category.iloc[:,0]=='ambiance']

In [10]:
expanded_df = pd.concat((df3_count,extrasample_df),axis=0)

In [11]:
expanded_df.reset_index(drop=True,inplace=True)

In [18]:
expanded_df.iloc[:,1].value_counts()

food        10292
overall      6534
others       3446
service      3074
ambiance     2940
Name: category, dtype: int64

## Count Vector

In [ ]:
##Split into train and test at 75/25
train, test = train_test_split(expanded_df.values,test_size = 0.25, random_state=1)

##Split X & Y
X_train = train[:,2:]
Y_train = train[:,1]
X_test = test[:,2:]
Y_test = test[:,1]

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## NaiveBayes
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train,Y_train)
nb_Y_pred = nb.predict(X_test)
print("NB Accuracy: ",np.mean(nb_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,nb_Y_pred))

## Logistic
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(multi_class='multinomial',solver='newton-cg').fit(X_train,Y_train)
log_Y_pred = log.predict(X_test)
print("Logistc Accuracy: ",np.mean(log_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,log_Y_pred))

##KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5).fit(X_train,Y_train)
knn_Y_pred = knn.predict(X_test)
print(neigh,"KNN Accuracy: ",np.mean(knn_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,knn_Y_pred))

## Linear SVC
from sklearn.svm import SVC
svcl = SVC(kernel='linear').fit(X_train,Y_train)
svcl_Y_pred = svcl.predict(X_test)
print("SVC Linear Accuracy: ",np.mean(svcl_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,svcl_Y_pred))

## RBF SVC
# from sklearn.linear_model import SGDClassifier
# sgd = SGDClassifier(loss='perceptron',penalty='elasticnet',l1_ratio=0.5).fit(X_train,Y_train)
# sgd_Y_pred = sgd.predict(X_test)
# print("SGD Perceptron Accuracy: ",np.mean(sgd_Y_pred == np.array(Y_test)))
# print(classification_report(Y_test,sgd_Y_pred))

## Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier().fit(X_train,Y_train)
dt_Y_pred = dt.predict(X_test)
print("Decision Tree Accuracy: ",np.mean(dt_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,dt_Y_pred))

## Random Forests
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train,Y_train)
rf_Y_pred = rf.predict(X_test)
print("Random Forests Accuracy: ",np.mean(rf_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,rf_Y_pred))

NB Accuracy:  0.627358490566
             precision    recall  f1-score   support

   ambiance       0.65      0.64      0.64       726
       food       0.71      0.77      0.74      2570
     others       0.45      0.35      0.39       897
    overall       0.55      0.54      0.54      1656
    service       0.63      0.65      0.64       723

avg / total       0.62      0.63      0.62      6572



## TFIDF 

In [ ]:
##Split into train and test at 75/25
train, test = train_test_split(df3_count.values,test_size = 0.25, random_state=1)

##Split X & Y
X_train = train[:,2:]
Y_train = train[:,1]
X_test = test[:,2:]
Y_test = test[:,1]

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## NaiveBayes
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train,Y_train)
nb_Y_pred = nb.predict(X_test)
print("NB Accuracy: ",np.mean(nb_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,nb_Y_pred))

## Logistic
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(multi_class='multinomial',solver='newton-cg').fit(X_train,Y_train)
log_Y_pred = log.predict(X_test)
print("Logistc Accuracy: ",np.mean(log_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,log_Y_pred))

##KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5).fit(X_train,Y_train)
knn_Y_pred = knn.predict(X_test)
print(neigh,"KNN Accuracy: ",np.mean(knn_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,knn_Y_pred))

## Linear SVC
from sklearn.svm import SVC
svcl = SVC(kernel='linear').fit(X_train,Y_train)
svcl_Y_pred = svcl.predict(X_test)
print("SVC Linear Accuracy: ",np.mean(svcl_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,svcl_Y_pred))

## RBF SVC
# from sklearn.linear_model import SGDClassifier
# sgd = SGDClassifier(loss='perceptron',penalty='elasticnet',l1_ratio=0.5).fit(X_train,Y_train)
# sgd_Y_pred = sgd.predict(X_test)
# print("SGD Perceptron Accuracy: ",np.mean(sgd_Y_pred == np.array(Y_test)))
# print(classification_report(Y_test,sgd_Y_pred))

## Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier().fit(X_train,Y_train)
dt_Y_pred = dt.predict(X_test)
print("Decision Tree Accuracy: ",np.mean(dt_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,dt_Y_pred))

## Random Forests
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train,Y_train)
rf_Y_pred = rf.predict(X_test)
print("Random Forests Accuracy: ",np.mean(rf_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,rf_Y_pred))

In [ ]:
##Pickle models
from sklearn.externals import joblib
joblib.dump(nb,'nb_sentence.pkl')
joblib.dump(knn,'knn_sentence.pkl')
joblib.dump(log,'log_sentence.pkl')
joblib.dump(svcl,'svcl_sentence.pkl')
joblib.dump(sgd,'sgd_sentence.pkl')
joblib.dump(dt,'dt_sentence.pkl')
joblib.dump(rf,'rf_sentence.pkl')

# Neural Network

In [ ]:
# food = 1
# service = 2
# ambiance = 3
# overall = 4
# others = 5

def integerize(x):
    if x == 'food':
        return 1
    elif x == 'service':
        return 2
    elif x == 'ambiance':
        return 3
    elif x == 'overall':
        return 4
    else:
        return 5

In [ ]:
df4 = df3_tf.copy()

In [ ]:
df4.category = df3_tf.category.apply(integerize)

In [ ]:
train_nn, test_nn = train_test_split(df4.values,test_size = 0.25)
X_train_nn = train_nn[:,2:]
Y_train_nn = train_nn[:,1]
X_test_nn = test_nn[:,2:]
Y_test_nn = test_nn[:,1]

In [ ]:
from sknn.mlp import Classifier, Layer

nn = Classifier(
    layers=[
        Layer("Tanh", units=100),
        Layer("Softmax")],
    learning_rate=0.001,
    n_iter=50,
    loss_type='mcc')
nn.fit(df4.iloc[:,2:], df4.iloc[:,1])

In [ ]:
y_pred = nn.predict(df4.iloc[:,2:])

In [ ]:
print("NN: ",np.mean(y_pred == np.array(df4.iloc[:,1])))
print(classification_report(df4.iloc[:,1],y_pred))

# Vectorizing New Text

In [42]:
new_text = df2.sentence

In [57]:
vocab_list = tfvectorizer.get_feature_names()

In [58]:
vocab_dict = dict.fromkeys(vocab_list)

In [59]:
tfvectorizer2 = TfidfVectorizer(ngram_range=(1,3),min_df = 3,tokenizer=tokenizeText,sublinear_tf=True,vocabulary=vocab_dict)

In [60]:
len(vocab_list)

208

In [68]:
check = tfvectorizer.transform(new_text[0:100])

In [69]:
check

<100x208 sparse matrix of type '<class 'numpy.float64'>'
	with 286 stored elements in Compressed Sparse Row format>